In [ ]:
import os
from openai import AzureOpenAI, OpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from asyncio.__main__ import key


client = AzureOpenAI(
    api_version="2025-01-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_KEY"),
)

# client = OpenAI(
#     api_key=os.getenv("OPENAI_API_KEY")
# )

In [42]:
def chat(messages):
    response = client.chat.completions.create(
        messages=messages,
        max_tokens=16384,
        temperature=1.0,
        top_p=1.0,
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    )
    return response.choices[0].message.content

In [43]:
def chat_with_history(history, user_input):
    messages = history + [{
        "role": "user",
        "content": user_input,
    }]
    response = chat(messages)
    messages.append({
        "role": "assistant",
        "content": response,
    })
    return messages, response

In [46]:
history = [
    {"role": "system", "content": "You are a helpful assistant that tells jokes in poem."},
]

history, response = chat_with_history(history, "tell me a joke")
history, response = chat_with_history(history, "tell me a another joke")

In [47]:
history, response = chat_with_history(history, "tell me a third joke")
print(response)

Sure thing! Here’s your third joke — short, snappy, and still in rhyme:  

---

I told my fridge a clever pun,  
It hummed and laughed — I guess I won.  
But now it won’t stop making cheer...  
Guess it’s just *running on cool* this year!  

---

Want me to make your **fourth joke a bit absurd and wacky**?
